In [ ]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
#comments are in english by obvious reasons. If someone would like to make comments, please, write in english.

# Интерполяция многочленами

Пусть задана сетка интерполяции (последовательность узлов $\{x_i\}_{i=0}^{n-1}$ со значениями функции $f(x_i)=f_i$). Задача интерполяции(многочленами) состоит в том, чтобы найти такую функцию(многочлен) $L(x)$, что во всех узлах $L(x_i)=f(x_i)$.

## Интерполяционный многочлен

Есть две популярных формы записи интерполяционного многочлена: форма Лагранжа и форма Ньютона. Вторую вычислять проще, поэтому мы в этом семинаре будем пользоваться ей. Для заданной сетки интерполяции многочлен Ньютона
$$
L(x) = f(x_0) + f_\Delta(x_0;x_1)(x-x_0) + ... + f_\Delta^n(x_0;..;x_{n-1})(x-x_0)\cdot...\cdot(x-x_{n-2}).
$$
Для вычисления значения необходимо заранее вычислить коэффициенты, включающие в себя разделённые разности. Реализуем многочлен в виде класса с функцией compute(x), возвращающей значение многочлена Ньютона в точке $x$.

In [ ]:
class NewtonPolynomial:
    #
    def __init__(self,xGrid,fs):
        self.xGrid = xGrid
        self.fs = fs
        
        self.ComputeCoefs(self.xGrid,self.fs)
    
    def ComputeDifferences(self):
        '''
            Computes differences f(x_0; x_1; x_2; x_3)
        '''
        self.diffs = [self.fs] #0th order difference
        #self.diffs is a list of differences of varyig order
        #self.diffs[1] is the list of differences of order 1

        #higher orders
        for i in np.arange(1,len(self.fs)):
            
            diff = #???
            self.diffs = self.diffs + [diff]
            
        #print(self.diffs)
        
        
    def ComputeCoefs(self,xs,fs):
        '''
            Extracts coefficients from the differences
            Input
            float[] xs -- argument grid (N,)
            float[] fs -- function values (N,)
        '''
        self.ComputeDifferences()
        
        self.coefs = #??
        
        #print(self.coefs)
        
        
    def Compute(self, x):
        '''
            Computes value of the interpolation at point(s) x
            Input
            float[] x -- (array of) points (N,) or (1,)
        '''
        if(x.shape[0]>1):
            vals = np.cumprod( x[:,None] - self.xGrid[None,:], axis=-1)[:,:-1]
        else:
            vals = np.cumprod( x - self.xGrid[None,:], axis=-1)[:,:-1]
            
        #print('computing')
        vals = #???
        #print(vals.shape)
        #print(self.coefs.shape)
        #print(x)
        
        return vals@self.coefs #RESULT     
            

Протестируем.

In [ ]:
def f(x):
    return  np.log(5*x) - x**2 + 5*np.cos(3*x)

nKnots=6
xZero=1
xLast=5
xs = xZero + np.arange(nKnots)*(xLast-xZero)/(nKnots-1)
fs = f(xs)

newtPoly = NewtonPolynomial(xs,fs)

In [ ]:
nKnotsDense =  600
xDense = xZero + np.arange(nKnotsDense)*(xLast-xZero)/(nKnotsDense-1)
interpF = newtPoly.Compute(xDense)
print(interpF.shape, xDense.shape)

fig,ax = plt.subplots(figsize=(7,7))

ax.grid()
ax.set_title('Comparison of Interpolation and Function',fontsize=16)
ax.set_xlabel('X',fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=13)
ax.plot(xDense,f(xDense))
ax.scatter(xs,fs)

ax.plot(xDense,interpF)
#f(xDense)
ax.legend(['Ground Truth','Interpolation','Knots'])

Попробуем поменять период косинуса в выражении для функции $f$.

In [ ]:
def fa(x,a):
    return  np.log(5*x) - x**2 + 5*np.cos(a*x)

periods = np.array([1,3,5,10])



nKnots=6
xZero=1
xLast=5
xs = xZero + np.arange(nKnots)*(xLast-xZero)/(nKnots-1)
nKnotsDense =  600
xDense = xZero + np.arange(nKnotsDense)*(xLast-xZero)/(nKnotsDense-1)


def runTest(periods,fa,xs,xDense):
    
    f,ax = plt.subplots(1,len(periods),figsize=(20,5))
    ax=list(ax)
    for k in np.arange(len(periods)):
        
        fs = fa(xs,periods[k])
        fDense = fa(xDense,periods[k])
        
        newtPoly = NewtonPolynomial(xs,fs)
        interpF = newtPoly.Compute(xDense)
        
        #plots
        ax[k].grid()
        ax[k].set_title(r'$a='\
                        +"{:.2f}".format(periods[k])+'$',fontsize=16)
        ax[k].set_xlabel('X',fontsize=14)
        ax[k].tick_params(axis='both', which='major', labelsize=13)
        ax[k].plot(xDense,fa(xDense,periods[k]))
        ax[k].scatter(xs,fs)

        ax[k].plot(xDense,interpF)
        #f(xDense)
        ax[k].legend(['Ground Truth','Interpolation','Knots'])
    
    f.tight_layout(pad=1.0)
        

runTest(periods,fa,xs,xDense)

## Интерполяция не обязательно сходится при росте числа узлов

Рассмотрим относительно простую функцию $f(x)= |x|$, теорема об остаточном члене погрешности не работает в окрестности нуля из-за разрыва производной. Поэтому возникают проблемы.

In [ ]:
def f(x):
    return np.abs(x)

nKnots=7 #see what happens if it is greater  than 15!
xZero=-2
xLast=2
xs = xZero + np.arange(nKnots)*(xLast-xZero)/(nKnots-1)
fs = f(xs)

newtPoly = NewtonPolynomial(xs,fs)

nKnotsDense =  600
xDense = xZero + np.arange(nKnotsDense)*(xLast-xZero)/(nKnotsDense-1)
interpF = newtPoly.Compute(xDense)
print(interpF.shape, xDense.shape)

fig,ax = plt.subplots(figsize=(7,7))

ax.grid()
ax.set_title('Comparison of Interpolation and Function',fontsize=16)
ax.set_xlabel('X',fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=13)
ax.plot(xDense,f(xDense))
ax.scatter(xs,fs)

ax.plot(xDense,interpF)
#f(xDense)
ax.legend(['Ground Truth','Interpolation','Knots'])

# Более практическая задача

GPS считывает положение смартфона каждые 5 секунд, определяя при этом координаты (для простоты предположим, что наплоскости, а не на сфере) с точностью до 2 метров (иногда сигнал ловит лучше и получется точнее). В результате одной велопоездки были получены данные о положении велосипеда. Найдите промежуточные положения велосипедиста и вычислите приблизительно его скорость в каждой точке маршрута. Нарисуйте маршрут на плоскости с указанием скорости.

## Решение

Воспользуемся уже написанными процедурами для интерполяции, чтобы уточнить маршрут. Чтобы сократить вычисления, лучше использовать кусочно-многочленную интерполяцию.

In [ ]:
with open('./tripDataNoNoise.pkl','rb') as f:
    dic=pickle.load(f)
    xs=dic['xs']
    ys=dic['ys'] 
    ts=dic['ts']

f,ax = plt.subplots(figsize=(7,7))

ax.grid()
ax.plot(xs,ys)

In [ ]:
newtPolysX = []
newtPolysY = []
knots=3 #take each 5 knots


for k in np.arange(0,len(xs),knots):
    tGrid = np.array(ts[k:(k+knots+1)])
    xGrid = xs[k:(k+knots+1)]
    yGrid = ys[k:(k+knots+1)]
    newtPolyX = NewtonPolynomial(tGrid,xGrid)
    newtPolyY = NewtonPolynomial(tGrid,yGrid)
    
    newtPolysX = newtPolysX + [newtPolyX]
    newtPolysY = newtPolysY + [newtPolyY]

In [ ]:
#Let's Plot!
tDense = np.arange(0,3600)
interpCurveX = np.zeros([len(tDense)])
interpCurveY = np.zeros([len(tDense)])

polyId=0
for t in np.arange(len(tDense)):
    if( tDense[t]>ts[(polyId+1)*knots]):
        polyId=polyId+1
        
    #print('t',t,'polyId',polyId)
    interpCurveX[t]= newtPolysX[polyId].Compute( np.array([tDense[t]]) )
    interpCurveY[t]= newtPolysY[polyId].Compute( np.array([tDense[t]]) )
        

In [ ]:
newtPolysX[0].coefs

In [ ]:
f,ax = plt.subplots(figsize=(17,17))

ax.grid()
ax.scatter(xs[:50],ys[:50])
ax.plot(interpCurveX[:170],interpCurveY[:170])


На незашумлённом интерполяция получается очень качественной. Попробуйте поэкспериментировать с уровнем шума в симуляторе.

In [ ]:
with open('./tripData.pkl','rb') as f:
    dic=pickle.load(f)
    xs=dic['xs']
    ys=dic['ys'] 
    ts=dic['ts']

f,ax = plt.subplots(figsize=(7,7))

ax.grid()
ax.plot(xs,ys)

In [ ]:
newtPolysX = []
newtPolysY = []
knots=3 #take each 5 knots


for k in np.arange(0,len(xs),knots):
    tGrid = np.array(ts[k:(k+knots+1)])
    xGrid = xs[k:(k+knots+1)]
    yGrid = ys[k:(k+knots+1)]
    newtPolyX = NewtonPolynomial(tGrid,xGrid)
    newtPolyY = NewtonPolynomial(tGrid,yGrid)
    
    newtPolysX = newtPolysX + [newtPolyX]
    newtPolysY = newtPolysY + [newtPolyY]

In [ ]:
#Let's Plot!
tDense = np.arange(0,3600)
interpCurveX = np.zeros([len(tDense)])
interpCurveY = np.zeros([len(tDense)])

polyId=0
for t in np.arange(len(tDense)):
    if( tDense[t]>ts[(polyId+1)*knots]):
        polyId=polyId+1
        
    #print('t',t,'polyId',polyId)
    interpCurveX[t]= newtPolysX[polyId].Compute( np.array([tDense[t]]) )
    interpCurveY[t]= newtPolysY[polyId].Compute( np.array([tDense[t]]) )

In [ ]:
f,ax = plt.subplots(figsize=(17,17))

ax.grid()
ax.scatter(xs[:50],ys[:50])
ax.plot(interpCurveX[:170],interpCurveY[:170])


## Выводы

Интерполяция не очень хороша, когда данные зашумлены, так как шумы добавляют много в ошибку. К этой задаче мы ещё вернёмся, когда будем говорить про метод наименьших квадратов и его обобщения.

Интерполяция многочленами -- самая простая и сейчас не представляет большого практического интереса, существуют и другие базисы, см. например Фурье-интерполяцию, сплайны, интерполяцию рациональными функциями.